In [ ]:
!pip install plotly

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import plotly.graph_objects as go

def visualize_data_flow_through_llm(model, tokenizer, input_text):
    """
    Visualize how the input transforms through the layers of the LLM.
    """
    # Step 1: Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt")
    input_ids = inputs["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Store intermediate representations
    representations = []
    layer_labels = []

    # Step 2: Pass through embedding layer
    embedding_output = model.embeddings(input_ids)
    representations.append(embedding_output[0].detach().numpy())
    layer_labels.append("Embeddings")

    # Step 3: Pass through each transformer layer
    hidden_states = embedding_output
    for i, layer in enumerate(model.transformer.layer):  # Correct access for DistilBERT layers
        hidden_states = layer(hidden_states)[0]  # Process one transformer block
        representations.append(hidden_states[0].detach().numpy())
        layer_labels.append(f"Transformer Layer {i + 1}")

    # Step 4: Pass through the output layer
    final_output = model(inputs["input_ids"]).last_hidden_state
    representations.append(final_output[0].detach().numpy())
    layer_labels.append("Output")

    # Step 5: Create a 3D visualization of the representations
    fig = go.Figure()

    for i, (rep, label) in enumerate(zip(representations, layer_labels)):
        # Reduce dimensionality to 3D using PCA
        from sklearn.decomposition import PCA
        pca = PCA(n_components=3)
        reduced_rep = pca.fit_transform(rep)

        # Add the layer's representation as a scatter plot
        fig.add_trace(go.Scatter3d(
            x=reduced_rep[:, 0],
            y=reduced_rep[:, 1],
            z=reduced_rep[:, 2],
            mode='markers',
            name=label,
            marker=dict(size=5),
            text=[f"Token: {token}" for token in tokens]
        ))

    # Set up the layout
    fig.update_layout(
        title="Data Flow Through LLM Layers",
        scene=dict(
            xaxis_title="PCA Component 1",
            yaxis_title="PCA Component 2",
            zaxis_title="PCA Component 3"
        ),
        width=1000,
        height=800
    )

    fig.show()

def main():
    # Load a small pre-trained model and tokenizer
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()  # Set the model to evaluation mode

    # Input text for demonstration
    input_text_1 = "college university school party president"
    input_text_2 = "nation country district region lifeboat"
    input_text_3 = "cow boat plane grass name ghost"

    # Visualize data flow
    visualize_data_flow_through_llm(model, tokenizer, input_text_1)
    visualize_data_flow_through_llm(model, tokenizer, input_text_2)
    visualize_data_flow_through_llm(model, tokenizer, input_text_3)

if __name__ == "__main__":
    main()


In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def visualize_data_flow_through_llm(model, tokenizer, input_text):
    """
    Visualize how the input transforms through the layers of the LLM.
    Also calculates and displays cosine similarity between specific words.
    """
    # Step 1: Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt")
    input_ids = inputs["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Words of interest (exclude '[CLS]' and '[SEP]')
    words_of_interest = ["college", "university", "school", "party", "president"]

    # Find tokenized versions of the words of interest
    indices_of_interest = []
    for word in words_of_interest:
        word_tokens = tokenizer.tokenize(word)
        for token in word_tokens:
            # Find all indices of the tokens that match subword pieces of the word of interest
            indices_of_interest.extend([i for i, t in enumerate(tokens) if t == token])

    # Ensure there are valid indices to calculate similarity
    if not indices_of_interest:
        print("No matching tokens found for the words of interest.")
        return

    # Store intermediate representations
    representations = []
    layer_labels = []

    # Step 2: Pass through embedding layer
    embedding_output = model.embeddings(input_ids)
    representations.append(embedding_output[0].detach().numpy())
    layer_labels.append("Embeddings")

    # Step 3: Pass through each transformer layer
    hidden_states = embedding_output
    for i, layer in enumerate(model.transformer.layer):  # Correct access for DistilBERT layers
        hidden_states = layer(hidden_states)[0]  # Process one transformer block
        representations.append(hidden_states[0].detach().numpy())
        layer_labels.append(f"Transformer Layer {i + 1}")

    # Step 4: Pass through the output layer
    final_output = model(inputs["input_ids"]).last_hidden_state
    representations.append(final_output[0].detach().numpy())
    layer_labels.append("Output")

    # Step 5: Calculate Cosine Similarity between words of interest at the embedding layer
    embeddings = representations[0]  # Get embeddings at the embedding layer (first layer)

    # Extract embeddings for the words of interest
    word_embeddings = embeddings[indices_of_interest]  # Extract embeddings for the words of interest

    # Ensure the embeddings are in the correct shape for cosine_similarity (2D array)
    similarity_matrix = cosine_similarity(word_embeddings)

    # Print Cosine Similarity for pairs of words
    print("Cosine Similarity Matrix (Words of Interest):")
    for i in range(len(indices_of_interest)):
        for j in range(i + 1, len(indices_of_interest)):
            word_1 = tokens[indices_of_interest[i]]
            word_2 = tokens[indices_of_interest[j]]
            print(f"Cosine similarity between '{word_1}' and '{word_2}': {similarity_matrix[i, j]:.4f}")

    # Step 6: Create a 3D visualization of the representations
    fig = go.Figure()

    for i, (rep, label) in enumerate(zip(representations, layer_labels)):
        # Reduce dimensionality to 3D using PCA
        pca = PCA(n_components=3)
        reduced_rep = pca.fit_transform(rep)

        # Add the layer's representation as a scatter plot
        fig.add_trace(go.Scatter3d(
            x=reduced_rep[:, 0],
            y=reduced_rep[:, 1],
            z=reduced_rep[:, 2],
            mode='markers',
            name=label,
            marker=dict(size=5),
            text=[f"Token: {token}" for token in tokens]
        ))

    # Set up the layout
    fig.update_layout(
        title="Data Flow Through LLM Layers",
        scene=dict(
            xaxis_title="PCA Component 1",
            yaxis_title="PCA Component 2",
            zaxis_title="PCA Component 3"
        ),
        width=1000,
        height=800
    )

    fig.show()

def main():
    # Load a small pre-trained model and tokenizer
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()  # Set the model to evaluation mode

    # Input text for demonstration
    input_text_1 = "college university school party president"
    input_text_2 = "nation country district region lifeboat"
    input_text_3 = "cow boat plane grass name ghost"

    # Visualize data flow and calculate cosine similarity
    visualize_data_flow_through_llm(model, tokenizer, input_text_1)
    visualize_data_flow_through_llm(model, tokenizer, input_text_2)
    visualize_data_flow_through_llm(model, tokenizer, input_text_3)

if __name__ == "__main__":
    main()


Cosine Similarity Matrix (Words of Interest):
Cosine similarity between 'college' and 'university': 0.5054
Cosine similarity between 'college' and 'school': 0.4356
Cosine similarity between 'college' and 'party': 0.0902
Cosine similarity between 'college' and 'president': 0.0624
Cosine similarity between 'university' and 'school': 0.3465
Cosine similarity between 'university' and 'party': 0.1272
Cosine similarity between 'university' and 'president': 0.0656
Cosine similarity between 'school' and 'party': 0.1541
Cosine similarity between 'school' and 'president': -0.0048
Cosine similarity between 'party' and 'president': 0.0397


No matching tokens found for the words of interest.
No matching tokens found for the words of interest.
